In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

In [72]:
path = "/Users/thrilok/Desktop/mantra_collab_job/work_files/latest_broko_code"
df = pd.read_csv(path  + "/Dataset/ML_CLEAN_DATA__Bdv2.7_RES.csv")
df.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181490 entries, 0 to 181489
Data columns (total 30 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ML_Number            181490 non-null  object 
 1   Area                 181490 non-null  object 
 2   Municipality         178872 non-null  object 
 3   Community            178872 non-null  object 
 4   Postal_Code          181359 non-null  object 
 5   Postal_Short         181359 non-null  object 
 6   Style                181490 non-null  object 
 7   Type_other           181490 non-null  int64  
 8   Type_detached        181490 non-null  int64  
 9   Type_SemiDetached    181490 non-null  int64  
 10  Cluster              181490 non-null  object 
 11  List_Price           181485 non-null  float64
 12  Cluster_Price        181490 non-null  float64
 13  Taxes                181488 non-null  float64
 14  Cluster_Tax          181490 non-null  float64
 15  Bedrooms         

In [73]:
# result =df[df['HPI_Sold_Price'] > 50000]
# result.info(verbose=True,show_counts=True)

In [74]:
df = df[df['Municipality'] == 'Mississauga']
# df = df[df['Community'] == 'lakeview']
df.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7578 entries, 2960 to 181428
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ML_Number            7578 non-null   object 
 1   Area                 7578 non-null   object 
 2   Municipality         7578 non-null   object 
 3   Community            7578 non-null   object 
 4   Postal_Code          7576 non-null   object 
 5   Postal_Short         7576 non-null   object 
 6   Style                7578 non-null   object 
 7   Type_other           7578 non-null   int64  
 8   Type_detached        7578 non-null   int64  
 9   Type_SemiDetached    7578 non-null   int64  
 10  Cluster              7578 non-null   object 
 11  List_Price           7578 non-null   float64
 12  Cluster_Price        7578 non-null   float64
 13  Taxes                7578 non-null   float64
 14  Cluster_Tax          7578 non-null   float64
 15  Bedrooms             7578 non-nul

In [75]:
df_thres = df[df['HPI_Sold_Price'] > 50000]
drop_cols = ["ML_Number","Postal_Code","Sold_Price","Month_Year","HPI_for_Month","Municipality","Area","Cluster","List_Price","Days_On_Market","lat","lng","Community","Basement1","Postal_Short"]
# drop_cols = ['ML_Number','Sold_Date',"Address","Area","Postal_Code","Air_Conditioning","Exterior1","Rooms","Tax_Year","Water_Included","property_type"]#,"lat","lng"]
data = df_thres.drop(drop_cols, axis=1)
data.isnull().sum()

Style                  0
Type_other             0
Type_detached          0
Type_SemiDetached      0
Cluster_Price          0
Taxes                  0
Cluster_Tax            0
Bedrooms               0
Washrooms              0
Basement_finished      0
Basement_unfinished    0
Basement_partial       0
Exterior1              3
Garage_Type            0
HPI_Sold_Price         0
dtype: int64

In [76]:
data = data.dropna(axis = 0, how ='any') 

In [77]:
q1 = data['HPI_Sold_Price'].quantile(0.25)
q3 = data['HPI_Sold_Price'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
lower_bound_outliers = data[data['HPI_Sold_Price'] < lower_bound] 
upper_bound_outliers = data[data['HPI_Sold_Price'] > upper_bound]
# lower_bound_outliers.to_csv(path + '/Dataset/lower_bound_outliers.csv')
# upper_bound_outliers.to_csv(path +'/Dataset/upper_bound_outliers.csv')    
dataset = data[(data['HPI_Sold_Price'] >= lower_bound) & (data['HPI_Sold_Price'] <= upper_bound)]

In [78]:
dataset.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7091 entries, 2960 to 181428
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Style                7091 non-null   object 
 1   Type_other           7091 non-null   int64  
 2   Type_detached        7091 non-null   int64  
 3   Type_SemiDetached    7091 non-null   int64  
 4   Cluster_Price        7091 non-null   float64
 5   Taxes                7091 non-null   float64
 6   Cluster_Tax          7091 non-null   float64
 7   Bedrooms             7091 non-null   int64  
 8   Washrooms            7091 non-null   int64  
 9   Basement_finished    7091 non-null   int64  
 10  Basement_unfinished  7091 non-null   int64  
 11  Basement_partial     7091 non-null   int64  
 12  Exterior1            7091 non-null   object 
 13  Garage_Type          7091 non-null   int64  
 14  HPI_Sold_Price       7091 non-null   float64
dtypes: float64(4), int64(9), object(2

In [79]:
X = dataset.drop('HPI_Sold_Price',axis=1)
y = dataset['HPI_Sold_Price']

In [80]:
# categorical features
categorical_features = [column for column, dtype in X.dtypes.items() if dtype==object]


In [81]:
model = LinearRegression()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Encode categorical features before training
label_encoders = {}
for feature in categorical_features:
    le = LabelEncoder()
    X_train[feature] = le.fit_transform(X_train[feature])
    X_test[feature] = le.fit_transform(X_test[feature])
    label_encoders[feature] = le


model.fit(
    X_train,y_train
)

y_pred = model.predict(X_test)

MAPE(y_test, y_pred)

0.10638717219187835

In [82]:
# Get the coefficients
coefficients = model.coef_

print(f"intercept= {model.intercept_}")

for feature,coefficients in zip(X.columns,coefficients):
    print(f"{feature} = {coefficients.astype(int)}")

intercept= -328150.34344688826
Style = 16622
Type_other = -12070
Type_detached = 70481
Type_SemiDetached = -58410
Cluster_Price = 0
Taxes = 1
Cluster_Tax = -10
Bedrooms = 142690
Washrooms = 118498
Basement_finished = 642
Basement_unfinished = 49881
Basement_partial = 0
Exterior1 = 12161
Garage_Type = 10324


In [ ]:
intercept= -328150.3434496727
Style = 16622
Type_other = -12070
Type_detached = 70481
Type_SemiDetached = -58410
Cluster_Price = 0
Taxes = 1
Cluster_Tax = -10
Bedrooms = 142690
Washrooms = 118498
Basement_finished = 2143
Basement_unfinished = 49881
Basement_partial = 0
Exterior1 = 12161
Garage_Type = 10324



In [13]:
feats = X_test.copy()
feats

,Community,Postal_Short,Style,Type_other,Type_detached,Type_SemiDetached,Taxes,Bedrooms,Washrooms,Basement1,Basement_finished,Basement_unfinished,Basement_partial,Exterior1,Garage_Type
109575,18,15,2,0,1,0,4849.000000,4,4,0,3,0,0,0,3
176802,22,9,2,0,0,1,5546.942256,4,5,3,0,1,0,2,3
72756,2,14,2,0,1,0,6055.000000,4,4,0,3,0,0,2,1
103140,13,3,2,0,1,0,8102.000000,4,3,0,3,0,0,6,3
102848,13,8,7,0,1,0,4529.270000,2,2,0,3,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107610,16,0,7,0,1,0,4189.660000,3,3,0,3,0,0,2,3
167951,3,14,2,0,0,1,4593.520000,3,4,3,0,1,0,2,3
126683,23,1,2,0,1,0,4842.470000,3,4,0,3,0,0,2,2
174893,21,5,4,0,0,1,4159.870000,3,2,3,0,1,0,2,3


In [50]:
cost_community = model.coef_[0] * feats["Community"]
cost_Postal_Short = model.coef_[1] * feats["Postal_Short"]
cost_Style = model.coef_[2] * feats["Style"]
cost_Type_other= model.coef_[3] * feats["Type_other"]
cost_Type_detached= model.coef_[4] * feats["Type_detached"]
cost_Type_SemiDetached= model.coef_[5] * feats["Type_SemiDetached"]
cost_Taxes= model.coef_[6] * feats["Taxes"]
cost_Bedrooms= model.coef_[7] * feats["Bedrooms"]
cost_Washrooms= model.coef_[8] * feats["Washrooms"]
cost_Basement1= model.coef_[9] * feats["Basement1"]
cost_Exterior1= model.coef_[10] * feats["Exterior1"]
cost_Garage_Type= model.coef_[11] * feats["Garage_Type"]

In [51]:
# man_predictions = model.intercept_ + cost_Postal_Short+cost_Style+cost_Type+cost_Taxes+cost_Bedrooms+cost_Washrooms+cost_Basement1+cost_Exterior1+cost_Garage_Type

# man_predictions.astype(int)

In [52]:
for col, le in label_encoders.items():
    X_test[col] = le.inverse_transform(X_test[col])

In [53]:
out = X_test
out['ML_Number'] = df['ML_Number']
out['Month_Year'] = df["Month_Year"]
out['HPI_for_Month'] = df["HPI_for_Month"]
out['Sold_Price'] = df['Sold_Price']
out['HPI_Sold_Price'] = df['HPI_Sold_Price']
out['Predicted_HPI_Price']  = y_pred.astype(int)
out['Diff']  = abs(out['HPI_Sold_Price'] - out['Predicted_HPI_Price']).astype(int)
out['Accuracy'] = (abs(1 - (abs(out['HPI_Sold_Price'] - out['Predicted_HPI_Price']))/out['HPI_Sold_Price']) * 100).astype(int)

# out['man_predictions'] = man_predictions.astype(int)
out

,Community,Postal_Short,Style,Type_other,Type_detached,Type_SemiDetached,Taxes,Bedrooms,Washrooms,Basement1,Exterior1,Garage_Type,ML_Number,Month_Year,HPI_for_Month,Sold_Price,HPI_Sold_Price,Predicted_HPI_Price,Diff,Accuracy
109575,meadowvale,l5n,2-storey,0,1,0,4849.000000,4,4,Finished,Alum Siding,3,W5445501,Dec 2021,364.0,1525000.0,1.475563e+06,1537962,62398,95
176802,port credit,l5g,2-storey,0,0,1,5546.942256,4,5,Finished,Brick,3,W6105140,Jul 2023,348.7,1875000.0,1.893820e+06,1403031,490788,74
72756,central erin mills,l5m,2-storey,0,1,0,6055.000000,4,4,Finished,Brick,1,W5520586,Mar 2022,391.9,2135000.0,1.918722e+06,1593191,325530,83
103140,lakeview,l4y,2-storey,0,1,0,8102.000000,4,3,Finished,Stone,3,W5585880,Apr 2022,379.5,2200000.0,2.041739e+06,1575554,466185,77
102848,lakeview,l5e,bungalow,0,1,0,4529.270000,2,2,Finished,Alum Siding,3,W5555799,Apr 2022,379.5,1100000.0,1.020870e+06,1118487,97617,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107610,malton,l4t,bungalow,0,1,0,4189.660000,3,3,Finished,Brick,3,W5421917,Dec 2021,364.0,1220000.0,1.180451e+06,1420202,239751,79
167951,churchill meadows,l5m,2-storey,0,0,1,4593.520000,3,4,Finished,Brick,3,W5447156,Dec 2021,364.0,1233000.0,1.193029e+06,1157083,35946,96
126683,rathwood,l4w,2-storey,0,1,0,4842.470000,3,4,Finished,Brick,2,W5490901,Feb 2022,392.7,1430888.0,1.283317e+06,1430109,146791,88
174893,mississauga valleys,l5a,Backsplit,0,0,1,4159.870000,3,2,Finished,Brick,3,W5495099,Feb 2022,392.7,1282000.0,1.149785e+06,943651,206133,82


In [54]:
out.insert(1,"cost_community",cost_community)
out.insert(3,"cost_Postal_Short",cost_Postal_Short)
out.insert(5,"cost_Style",cost_Style)
out.insert(7,"cost_Type_other",cost_Type_other)
out.insert(9,"cost_Type_detached",cost_Type_detached)
out.insert(11,"cost_Type_SemiDetached",cost_Type_SemiDetached)
out.insert(13,"cost_Taxes",cost_Taxes)
out.insert(15,"cost_Bedrooms",cost_Bedrooms)
out.insert(17,"cost_Washrooms",cost_Washrooms)
out.insert(19,"cost_Basement1",cost_Basement1)
out.insert(21,"cost_Exterior1",cost_Exterior1)
out.insert(23,"cost_Garage_Type",cost_Garage_Type)


out

,Community,cost_community,Postal_Short,cost_Postal_Short,Style,cost_Style,Type_other,cost_Type_other,Type_detached,cost_Type_detached,...,Garage_Type,cost_Garage_Type,ML_Number,Month_Year,HPI_for_Month,Sold_Price,HPI_Sold_Price,Predicted_HPI_Price,Diff,Accuracy
109575,meadowvale,-11785.178782,l5n,9938.551482,2-storey,40515.576109,0,-0.0,1,205161.207827,...,3,12247.935582,W5445501,Dec 2021,364.0,1525000.0,1.475563e+06,1537962,62398,95
176802,port credit,-14404.107400,l5g,5963.130889,2-storey,40515.576109,0,-0.0,0,0.000000,...,3,12247.935582,W6105140,Jul 2023,348.7,1875000.0,1.893820e+06,1403031,490788,74
72756,central erin mills,-1309.464309,l5m,9275.981383,2-storey,40515.576109,0,-0.0,1,205161.207827,...,1,4082.645194,W5520586,Mar 2022,391.9,2135000.0,1.918722e+06,1593191,325530,83
103140,lakeview,-8511.518009,l4y,1987.710296,2-storey,40515.576109,0,-0.0,1,205161.207827,...,3,12247.935582,W5585880,Apr 2022,379.5,2200000.0,2.041739e+06,1575554,466185,77
102848,lakeview,-8511.518009,l5e,5300.560790,bungalow,141804.516383,0,-0.0,1,205161.207827,...,3,12247.935582,W5555799,Apr 2022,379.5,1100000.0,1.020870e+06,1118487,97617,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107610,malton,-10475.714473,l4t,0.000000,bungalow,141804.516383,0,-0.0,1,205161.207827,...,3,12247.935582,W5421917,Dec 2021,364.0,1220000.0,1.180451e+06,1420202,239751,79
167951,churchill meadows,-1964.196464,l5m,9275.981383,2-storey,40515.576109,0,-0.0,0,0.000000,...,3,12247.935582,W5447156,Dec 2021,364.0,1233000.0,1.193029e+06,1157083,35946,96
126683,rathwood,-15058.839555,l4w,662.570099,2-storey,40515.576109,0,-0.0,1,205161.207827,...,2,8165.290388,W5490901,Feb 2022,392.7,1430888.0,1.283317e+06,1430109,146791,88
174893,mississauga valleys,-13749.375246,l5a,3312.850494,Backsplit,81031.152219,0,-0.0,0,0.000000,...,3,12247.935582,W5495099,Feb 2022,392.7,1282000.0,1.149785e+06,943651,206133,82


In [55]:
out.to_csv(path + "/Dataset/27-09-2023/out_linear_log_ga.csv",index=False)

## Logistic regression

In [162]:
from sklearn.linear_model import LogisticRegression

In [163]:
log_model = LinearRegression()
# Encode categorical features before training
label_encoders = {}
for feature in categorical_features:
    le = LabelEncoder()
    X_train[feature] = le.fit_transform(X_train[feature])
    X_test[feature] = le.fit_transform(X_test[feature])
    label_encoders[feature] = le


log_model.fit(
    X_train,y_train
)

y_pred_log = log_model.predict(X_test)

MAPE(y_test, y_pred_log)

/Users/thrilok/Documents/conda_envs/broko/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Accuracy
- Diff
- HPI_Sold_Price
- HPI_for_Month
- ML_Number
- ...

  warnings.warn(message, FutureWarning)


ValueError: could not convert string to float: 'W5445501'

In [276]:
print(f'model intercept: {log_model.intercept_}')

for feature,coefficient in zip(X.columns,log_model.coef_):
    print(f"{feature} = {coefficient.astype(int)}")

model intercept: 794657.4624137643
Postal_Short = 1857
Style = 7614
Type = -66687
Taxes = 12
Bedrooms = 36684
Washrooms = 186934
Basement1 = 1007
Exterior1 = 29059
Garage_Type = -73078
